# Avaliando a qualidade do modelo via cross validation
Esse notebook faz parte do material de apoio do tutorial [Introdução ao Scikit-learn - Parte 3: avaliando o modelo via cross validation](http://computacaointeligente.com.br/outros/intro-sklearn-part-3/)

In [ ]:
from sklearn.datasets import load_wine
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
import numpy as np

## Carregando os dados 

In [ ]:
wine_dataset = load_wine()
X = wine_dataset['data']
y = wine_dataset['target']
nome_das_classes = wine_dataset.target_names

## Criando o pipeline e executando N vezes
Antes de executarmos o *cross-validation* vamos executar o modelos `N` vezes. Mas dessa vez, sempre antes de treinar o modelo, vamos embaralhar a base (perceba que não setamos o parâmetro `random_state`. Por default ele fica como `None`).

Nesse experimento, calculamos a média da acurácia. Fica como exercício fazer o mesmo para a matriz de confusão, que foi calculada no parte 2.

In [ ]:
K = 3
N = 10
acuracias = list()
knn_pipeline = Pipeline(steps=[
  ("normalizacao", MinMaxScaler()),  
  ("KNN", KNeighborsClassifier(n_neighbors=K))
])

for i in range(N):
    X_treino, X_teste, y_treino, y_teste = train_test_split(X, y, test_size=0.3, shuffle=True)
    knn_pipeline.fit(X_treino, y_treino)
    ac_i = knn_pipeline.score(X_treino, y_treino)
    acuracias.append(ac_i)

print("- Acurácia:")
print(f"Media: {round(np.mean(acuracias) * 100, 2)}%")
print(f"Desvio padrão: {round(np.std(acuracias) * 100, 2)}%")

## Utilizando cross-validation
Ok, agora vamos utilizar cross-validation. Mas primeiro, precisamos do modelo base.

In [ ]:
knn_pipeline = Pipeline(steps=[
  ("normalizacao", MinMaxScaler()),  
  ("KNN", KNeighborsClassifier(n_neighbors=3))
])

### Usando `cross_val_score`
Essa função básicamente executa o cross-validation de acordo com o número de *folders* informada via o parâmetro `cv`. Na sequência ela retorna uma métrica de performance para cada folder de teste. Por padrão, essa métrica é a acurácia (para classificação). Mas podemos definir usando o parâmetro `scoring`

In [ ]:
acuracias = cross_val_score(knn_pipeline, X, y, cv=5)
print("acuracias:", acuracias)
print("acuracia final:", np.mean(acuracias), "+-", np.std(acuracias))

In [ ]:
f1s = cross_val_score(knn_pipeline, X, y, cv=5, scoring="f1_macro")
print("F1-macros:", f1s)
print("F1-macros:", np.mean(f1s), "+-", np.std(f1s))

### Usando `cross_validate`
A principal diferença para o método anterior é que podemos setar várias métricas. Além disso, ele sempre retorn o tempo de `fit` e de `score`

In [ ]:
nome_metricas = ['accuracy', 'precision_macro', 'recall_macro']
metricas = cross_validate(knn_pipeline, X, y, cv=5, scoring=nome_metricas)
for met in metricas:
    print(f"- {met}:")
    print(f"-- {metricas[met]}")
    print(f"-- {np.mean(metricas[met])} +- {np.std(metricas[met])}\n")    

## Gerando predições via `cross_val_predict`
Nesse caso, cada predição será obtida para o conjunto de teste de cada uma das *folders*. Em outras palavras, se `cv=5`, por exemplo, o modelo vai ser treinado para 4 partições e testado em 1, que gera as predições. Ao final das 5 execuções, os resultados são concatenados e retornados.

Podemos setar o parametro `method` para escolher qual predição será retornada.

**Observação**: os resultados para os dois métodos a seguir podem ser diferentes pois os modelos será retreinado para cada partição (que pode ser diferente). Vamos lidar com isso na sequência usando o `KFold`-like.

In [ ]:
pred = cross_val_predict(knn_pipeline, X, y, cv=5)
print(pred)

In [ ]:
pred_prob = cross_val_predict(knn_pipeline, X, y, cv=5, method="predict_proba")
print(pred_prob[0])

## Obtendo mais controle utilizando `KFold` e  `StratifiedKFold` 
Ambas funções retornam um *generator* com os indices para selecionar os dados de acordo com o número de *folders* informadas. A principal diferença é que o `StratifiedKFold` estratifica os dados de acordo com as classes. Em outras palavras, ele balanceia a quantidade de amostras de cada classe entre as *folders*. Isso é relevante se o dataset é desbalanceado, algo comum em ML

#### `KFold`

In [ ]:
n_folders = 5
cross_val = KFold(n_splits=n_folders, shuffle=True, random_state=32)
dados_cv = {f"folder_{f+1}": {"treino": None, "teste": None} for f in range(n_folders)}
k = 1
for indices_treino, indices_teste in cross_val.split(X):
    dados_cv[f"folder_{k}"]["treino"] = (X[indices_treino], y[indices_treino])
    dados_cv[f"folder_{k}"]["teste"] = (X[indices_teste], y[indices_teste])
    k+=1
print(dados_cv.keys())

As funções `cross_val_predict`, `cross_validate` e `cross_val_predict` aceitam que o parâmetro `cv` seja um inteiro ou um *generator* obtido através da `KFold`-like por exemplo. Isso garante que as partições sejam exatamente as mesmas e evita o problema citado quando usamos `method="predict_proba"`

In [ ]:
pred = cross_val_predict(knn_pipeline, X, y, cv=cross_val)
pred_prob = cross_val_predict(knn_pipeline, X, y, cv=cross_val, method="predict_proba")
print(pred[112])
print(pred_prob[112])

Também podemos tomar controle do loop de execução do modelo. A gente iniciou esse notebook executando o modelo`n` vezes para partições diferentes de treino e teste. Podemos fazer o mesmo, mas agora usando as *folders*. Isso é que as funções anteriores fazem por trás do pano. Mas por algum motivo, você pode querer controlar esse loop.

In [ ]:
acuracias = list()
for folder in dados_cv:       
    knn_pipeline.fit(dados_cv[folder]["treino"][0], dados_cv[folder]["treino"][1])
    ac_i = knn_pipeline.score(dados_cv[folder]["teste"][0], dados_cv[folder]["teste"][1])
    acuracias.append(ac_i)
    print(f"{folder}: {ac_i}")

print("\n- Acurácia das folders:")
print(f"Media: {round(np.mean(acuracias) * 100, 2)}%")
print(f"Desvio padrão: {round(np.std(acuracias) * 100, 2)}%")

#### `StratifiedKFold`
A ideia é basicamente a mesma da `KFold`, mas como vai ser estratificado por classe, precisamos passar `y` como parâmetro. Você pode substituir o *generator* obtido aqui para nas células anteriores

In [ ]:
n_folders = 5
cross_val_strat = StratifiedKFold(n_splits=n_folders, shuffle=True, random_state=32)
dados_cv_strat = {f"folder_{f+1}": {"treino": None, "teste": None} for f in range(n_folders)}
k = 1
for indices_treino, indices_teste in cross_val_strat.split(X, y):
    dados_cv[f"folder_{k}"]["treino"] = (X[indices_treino], y[indices_treino])
    dados_cv[f"folder_{k}"]["teste"] = (X[indices_teste], y[indices_teste])
    k+=1
print(dados_cv.keys())